In [ ]:
## https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [11]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import requests

In [10]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []

    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


In [3]:
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task = 'sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)


In [4]:
# download label mapping
labels = []
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


In [5]:
greatings = "Noor Bukhari, the popular Pakistani actress, has recently been blessed with a baby boy, Muhammad Ali Raza. This news has been creating a buzz in the media and among her fans. Noor, who is already a mother of two beautiful daughters, shared the happy news on her Instagram account with a picture of the newborn’s […] The post Actress Noor Bukhari’s Family Expands with the Arrival of a Baby Boy appeared first on Online English News - Latest Education, Business & Trending Enews."
tulip = "Suosikkisarjan viimeinen jakso nähdään Suomessa 29. toukokuuta."
church = "Syombua said Muendo left her house together with his wife and then two-and-a-half year-old son for Mackenzie’s church."
cryptocurrency = "Finance platform Wormhole has been hacked, and cryptocurrency worth more than $A449 million stolen."
nft_ger = "Die Popularität von künstlicher Intelligenz (KI) hat in diesem Jahr weltweit stark zugenommen. Zahlreiche Plattformen wie Openai’s Chatgpt 3.5, Chatgpt 4.0, DALL-E, Stable Diffusion sowie innovative Tools wie Midjourney und Google’s Bard sind derzeit ein großer Trend im Internet. Inmitten dieser technologischen Revolution hat Crypto.com CEO Kris Marszalek kürzlich eine Veranstaltung abgehalten, bei der er …"

In [ ]:
import requests

# Remplacez API_KEY par votre clé API si nécessaire
API_KEY = 'pub_21583f7c244644f662d18248c2be0396ebc44'

# Remplacez l'URL par l'URL de l'API que vous souhaitez appeler
url = 'https://newsdata.io/api/1/news?apikey={api_key}&q={val}'

# Définir les paramètres de requête si nécessaire
params = {
    'paramètre1': 'valeur1',
    'paramètre2': 'valeur2'
}

# Ajouter la clé API à la requête si nécessaire
headers = {
    'Authorization': f'Bearer {API_KEY}'
}

# Effectuer la requête GET et stocker la réponse dans la variable response
response = requests.get(url, params=params, headers=headers)

# Vérifier si la requête a réussi (code de statut 200 signifie succès)
if response.status_code == 200:
    # Imprimer la réponse au format JSON
    print(response.json())
else:
    # Afficher le code d'erreur si la requête a échoué
    print(f'Erreur {response.status_code}: {response.reason}')


In [3]:
#val = input('Enter a topic')

In [1]:
url_api = "https://newsdata.io/api/1/news?apikey={api_key}&q={val}" 

In [7]:
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "- Kazakhstan and Tajikistan signed a concept on the establishment of a joint venture, which will be located in Tajikistan and assist entrepreneurs of the two parties in the development of mutual trade, Kazinform reports with reference to Chairman of the Board of the Foreign Trade Chamber of Kazakhstan Ayan Yerenov. \"We have signed a concept with our colleagues to establish a joint venture in Tajikistan. This enterprise will assist entrepreneurs in the development of mutual trade, as well as in the development of trade with third countries,\" Yerenov said at the Kazakh-Tajik business forum. He added that, according to a preliminary analysis, Kazakh enterprises will be able to increase the additional volume of non-primary exports to Tajikistan worth of more than $170 million. This figure can grow to $2 billion in the next two years, the Chairman assured. \"We have already moved from the implementation of individual projects to a systemic partnership in the economy. This is evidenced by the agreements signed today for more than $1 billion,\" Yerenov noted. According to him, it is also planned to conduct a trade mission this year, where the target indicator will be $120 million. Great prospects for the development of cooperation exist in the petrochemical and pharmaceutical industries. Deputy Prime Minister of Tajikistan Sulaymon Ziezoda, speaking about the agreements with Kazakhstan reached at the business forum in Astana, said that an agreement was signed between the countries on the construction of wholesale distribution centers in order to increase the supply of fruits and vegetables that are in demand in Kazakhstan. He also expressed confidence that the issue of resuming air traffic between Dushanbe and Astana will be resolved."
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)


In [ ]:

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)


In [8]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


1) positive 0.734
2) neutral 0.2627
3) negative 0.0033
